In [1]:
import torch
import numpy as np
from scipy.spatial.distance import pdist, squareform
from scipy.linalg import block_diag
from scipy.stats import rankdata
import time

In [2]:
n_obs = 100

In [3]:
X = np.zeros((n_obs, 30))
coef_X = np.zeros((n_obs, 4))
xh1 = np.sin(np.linspace(0, 2 * np.pi, num=30))
xh2 = np.cos(np.linspace(0, 2 * np.pi, num=30))
xh3 = np.sin(2 * np.linspace(0, 2 * np.pi, num=30))
xh4 = np.cos(2 * np.linspace(0, 2 * np.pi, num=30))

for i in range(n_obs):
    coef_X[i, :] = np.array([np.random.normal(1, 1), np.random.normal(9, 1), np.random.normal(9, 1), np.random.normal(6, 1)])
    X[i, :] = coef_X[i, 0] * xh1 + coef_X[i, 1] * xh2 + coef_X[i, 2] * xh3 + coef_X[i, 3] * xh4

Y = np.zeros((n_obs, 30))
coef_Y = np.zeros((n_obs, 4))
yh1 = np.sin(np.linspace(0, 2 * np.pi, num=30))
yh2 = np.cos(np.linspace(0, 2 * np.pi, num=30))
yh3 = np.sin(2 * np.linspace(0, 2 * np.pi, num=30))
yh4 = np.cos(2 * np.linspace(0, 2 * np.pi, num=30))

for i in range(n_obs):
    coef_Y[i, :] = np.array([np.random.normal(2, 1), np.random.normal(0, 1), np.random.normal(2, 1), np.random.normal(3, 1)])
    Y[i, :] = coef_Y[i, 0] * yh1 + coef_Y[i, 1] * yh2 + coef_Y[i, 2] * yh3 + coef_Y[i, 3] * yh4


# Najszybsze podejście z R w Pythonie

In [4]:
def L_2(x):
    odl = pdist(x)
    sigma = np.median(odl)
    
    l_return = np.zeros((x.shape[0], x.shape[0]))
    const_1 = 2 * (sigma**2) / (1 + sigma**2)
    const_2 = 2 * (sigma**4) / (1 + sigma**2) + 2 * const_1
    
    norm_2 = squareform(odl**2)
    il_sk = np.dot(x, x.T)
    
    l_return = np.exp(-(norm_2 + const_1 * il_sk) / const_2)
    
    return l_return

def NDIME_3(X, Y, HH):
    return np.sum(np.diag(np.dot(np.dot(np.dot(L_2(X), HH), L_2(Y)), HH))) / X.shape[0]**2

n = n_obs
HH = np.eye(n) - np.outer(np.ones(n), np.ones(n)) / n

def perm_test_3(X, Y, B=1000):
    n = X.shape[0]
    test_statistic = NDIME_3(X, Y, HH)
    
    results = np.zeros(B)
    for i in range(B):
        Y_perm = Y[np.random.permutation(n), :]
        results[i] = NDIME_3(X, Y_perm, HH)
    
    p_value = np.mean(results > test_statistic)
    
    return p_value


In [5]:
np.random.seed(123)

tic = time.perf_counter()
results = perm_test_3(X, Y)
toc = time.perf_counter()
print(f"Execution time: {toc - tic} seconds")

Execution time: 0.7961181669961661 seconds


# GPU

In [6]:
def L_gpu(x):
    x = torch.tensor(x)
    odl = torch.cdist(x, x, p=2)
    mask = ~torch.triu(torch.ones(odl.size()), diagonal=0).bool()
    vector = torch.masked_select(odl, mask)
    sigma = torch.median(odl)
    
    l_return = torch.zeros((x.shape[0], x.shape[0]))
    const_1 = 2 * (sigma**2) / (1 + sigma**2)
    const_2 = 2 * (sigma**4) / (1 + sigma**2) + 2 * const_1
    
    norm_2 = odl**2
    il_sk = torch.matmul(x, x.T)
    
    l_return = torch.exp(-(norm_2 + const_1 * il_sk) / const_2)
    
    return l_return

def NDIME_gpu(X, Y, HH):
    return torch.sum(torch.diag(torch.matmul(torch.matmul(torch.matmul(L_gpu(X), HH), L_gpu(Y)), HH))) / X.shape[0]**2

n = n_obs
HH = np.eye(n) - np.outer(np.ones(n), np.ones(n)) / n
HH = torch.tensor(HH)

def perm_test_gpu(X, Y, B=1000):
    n = X.shape[0]
    test_statistic = NDIME_gpu(X, Y, HH)
    
    results = torch.zeros(B)
    for i in range(B):
        Y_perm = Y[np.random.permutation(n), :]
        results[i] = NDIME_gpu(X, Y_perm, HH)
    
    #p_value = torch.mean(results > test_statistic)
    p_value = torch.mean((results > test_statistic).to(torch.float))
    
    return p_value


In [7]:
np.random.seed(123)

tic = time.perf_counter()
results = perm_test_gpu(X, Y)
toc = time.perf_counter()
print(f"Execution time: {toc - tic} seconds")

Execution time: 0.6615312079957221 seconds
